In [39]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [40]:
data = pd.read_csv("Steel_industry_data.csv")
print(data.isnull().sum())  # Count of missing values in each column
# data['date'] = pd.to_datetime(data['date'])
# y_train=data['Usage_kWh'].values
data = data.drop(columns=['date'])
X_train=data.values

data.head()


date                                    0
Usage_kWh                               0
Lagging_Current_Reactive.Power_kVarh    0
Leading_Current_Reactive_Power_kVarh    0
CO2(tCO2)                               0
Lagging_Current_Power_Factor            0
Leading_Current_Power_Factor            0
NSM                                     0
WeekStatus                              0
Day_of_week                             0
Load_Type                               0
dtype: int64


,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [41]:
print(X_train)

[[3.17 2.95 0.0 ... 'Weekday' 'Monday' 'Light_Load']
 [4.0 4.46 0.0 ... 'Weekday' 'Monday' 'Light_Load']
 [3.24 3.28 0.0 ... 'Weekday' 'Monday' 'Light_Load']
 ...
 [3.78 3.17 0.07 ... 'Weekday' 'Monday' 'Light_Load']
 [3.78 3.06 0.11 ... 'Weekday' 'Monday' 'Light_Load']
 [3.67 3.02 0.07 ... 'Weekday' 'Monday' 'Light_Load']]


In [42]:
scaler = StandardScaler()

# Perform scaling and assign it after casting the columns to float
X_train[:, 1:7] = scaler.fit_transform(X_train[:, 1:7]).astype('float64')


print(X_train)
X_train.shape
# df=pd.DataFrame(X_train)
# df.to_csv('scaled_Steel_industry_data.csv', index=False, header=True)

[[3.17 -0.6185163432975694 -0.5213850478056785 ... 'Weekday' 'Monday'
  'Light_Load']
 [4.0 -0.5259110722465691 -0.5213850478056785 ... 'Weekday' 'Monday'
  'Light_Load']
 [3.24 -0.598278105253311 -0.5213850478056785 ... 'Weekday' 'Monday'
  'Light_Load']
 ...
 [3.78 -0.6050241846013972 -0.5119566216410291 ... 'Weekday' 'Monday'
  'Light_Load']
 [3.78 -0.6117702639494832 -0.5065689495469436 ... 'Weekday' 'Monday'
  'Light_Load']
 [3.67 -0.6142233837124237 -0.5119566216410291 ... 'Weekday' 'Monday'
  'Light_Load']]


(35040, 10)

In [43]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[7,8])],remainder='passthrough')
X_train=ct.fit_transform(X_train)
print(X_train)
print(X_train.shape)
# df=pd.DataFrame(X_train)
# df.to_csv('scaled_Steel_industry_data.csv', index=False, header=True)


[[1.0 0.0 0.0 ... 0.5132676163619359 -1.6780152604563736 'Light_Load']
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6419289107691397 'Light_Load']
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6058425610819058 'Light_Load']
 ...
 [1.0 0.0 0.0 ... 0.5126109334762843 1.6780152604563736 'Light_Load']
 [1.0 0.0 0.0 ... 0.5119542505906323 1.7141016101436073 'Light_Load']
 [1.0 0.0 0.0 ... 0.5126109334762843 -1.7141016101436073 'Light_Load']]
(35040, 17)


In [45]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X_train[:,-1]=le.fit_transform(X_train[:,-1])
print(X_train)
type(X_train)
df=pd.DataFrame(X_train)
df.to_csv('scaled_Steel_industry_data.csv', index=False, header=True)

[[1.0 0.0 0.0 ... 0.5132676163619359 -1.6780152604563736 0]
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6419289107691397 0]
 [1.0 0.0 0.0 ... 0.5132676163619359 -1.6058425610819058 0]
 ...
 [1.0 0.0 0.0 ... 0.5126109334762843 1.6780152604563736 0]
 [1.0 0.0 0.0 ... 0.5119542505906323 1.7141016101436073 0]
 [1.0 0.0 0.0 ... 0.5126109334762843 -1.7141016101436073 0]]


In [46]:
# Ensure data_scaled is converted to a NumPy array before splitting
node_data = np.array_split(X_train, 8)


In [47]:
global_model = SVR(kernel='linear')

In [48]:
def federated_averaging(svr_models):
    """Aggregate SVR models by averaging support vectors and coefficients."""
    # Collect support vectors and dual coefficients from all models
    all_support_vectors = np.concatenate([model.support_vectors_ for model in svr_models], axis=0)
    all_dual_coef = np.concatenate([model.dual_coef_ for model in svr_models], axis=0)

    # Average dual coefficients (may need custom logic based on your problem)
    avg_dual_coef = np.mean(all_dual_coef, axis=0)

    # Create a new SVR model to hold aggregated parameters
    aggregated_model = SVR(kernel='linear')  # Use the same kernel as your local models
    aggregated_model.support_vectors_ = all_support_vectors
    aggregated_model.dual_coef_ = avg_dual_coef
    return aggregated_model

# Function to train the local model on each node's data
def train_local_model(data):
    y = data[:, 11]  # Target (Usage_kWh)
    X = np.concatenate((data[:, :11], data[:, 12:]), axis=1)  # All columns except the target
    model = SVR(kernel='linear')  # Specify kernel type as needed
    model.fit(X, y)
    return model

In [49]:
# Federated Learning Process
for iteration in range(3):
    print(f"Iteration {iteration + 1}:")

    # Local models for each node
    local_models = []

    # Train each node's model using 1460 rows in this iteration
    for i, node in enumerate(node_data):
        start = iteration * 1460
        end = start + 1460
        node_subset = node[start:end]

        # Train the local model on the subset
        local_model = train_local_model(node_subset)
        local_models.append(local_model)

        # Evaluate the local model
        y_pred = local_model.predict(np.concatenate((node_subset[:, :11], node_subset[:, 12:]), axis=1))
        mse = mean_squared_error(node_subset[:, 11], y_pred)
        print(f"  Node {i + 1} - MSE: {mse:.4f}")

    # Aggregate the local model updates using Federated Averaging
    aggregated_model = federated_averaging(local_models)

    # Update the global model with aggregated parameters
    global_model.support_vectors_ = aggregated_model.support_vectors_
    global_model.dual_coef_ = aggregated_model.dual_coef_

    print(f"Global model updated after iteration {iteration + 1}.\n")

Iteration 1:
  Node 1 - MSE: 0.0895
  Node 2 - MSE: 0.1101
  Node 3 - MSE: 0.0959
  Node 4 - MSE: 0.0779
  Node 5 - MSE: 0.0972
  Node 6 - MSE: 0.0369
  Node 7 - MSE: 0.0252
  Node 8 - MSE: 0.0506


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 652 and the array at index 1 has size 538

In [36]:
# Final Evaluation (Optional): Use some test data for evaluation
test_data = node_data[0][:1460]  # Example: Using first 1460 rows from node 1 as test data
# X_test = test_data[:, 1:]
# y_test = test_data[:, 1]
test_data

y_test = test_data[:, 11]   # Target (Usage_kWh)
    # Select all columns except the 12th column using slicing
X_test= np.concatenate((test_data[:, :11], test_data[:, 12:]), axis=1)

df=pd.DataFrame(y_test)
df.to_csv('test_data.csv', index=False, header=True)

y_pred = global_model.predict(X_test)
final_mse = mean_squared_error(y_test, y_pred)
print(f"Final Global Model MSE on Test Data: {final_mse:.4f}")

AttributeError: 'RandomForestRegressor' object has no attribute 'estimators_'

In [12]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(r2)

0.8983988342396683
